In [1]:
%matplotlib inline
import os
import warnings

import critical_loads as cl
import gdal
import geopandas as gpd
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import numpy as np
import pandas as pd

warnings.simplefilter("ignore")
plt.style.use("ggplot")

In [2]:
# Connect to PostGIS
eng = nivapy.da.connect_postgis(database="critical_loads")

Connection successful.


# Critical loads: Water workflow using updated MAGIC inputs

This notebook calculates exceedances for 2012 - 2016, but using BC0 derived from two runs of the Magic model (1800 and 1860). Data from Magic are provided in the Excel templates here:

    shared/critical_loads/water_cl_input_templates
    
This workflow is closely based on the "standard" workflow [here](https://nbviewer.jupyter.org/github/JamesSample/critical_loads_2/blob/master/notebooks/04_water_new_grid.ipynb), but using some of the additional options in the Critical Loads functions to incorporate the Magic data. See the relevant `docstrings` and the e-amil from Kari received 02.11.2020 at 09.30 for details.   

## 1. Define deposition series and year of interest

Choose the deposition dataset and resolution you wish to work with (see notebook 01 for a list of available `series_ids`), and specify the year for the Magic data (1800 or 1860). Note that **S and N deposition grids at the specified resolution must already have been created in notebook 02** and saved here:

    shared/critical_loads/raster/deposition
    
120 m resolution is more than sufficient, since the highest resolution of raw data in this workflow is 0.1 degrees, which is several _kilometres_, even in northern Norway.

In [3]:
# Dep series of interest
ser_id = 28

# Choose cell size (30m, 60m or 120m)
cell_size = 120

# Year for Magic parameters (for calculating BC0_magic)
year = 1860

## 2. Calculate critical loads for water

In [4]:
# Read MAGIC data from spreadsheet
xl_path = f"/home/jovyan/shared/critical_loads/water_cl_input_templates/input_template_critical_loads_water_blr_magic_{year}.xlsx"
mag_df = pd.read_excel(xl_path, sheet_name="magic_parameters")

# Rasterise critical loads for water
out_fold = f"/home/jovyan/shared/critical_loads/raster/magic_{year}/water"
cl.rasterise_water_critical_loads(
    eng, out_fold, cell_size, bc0="BC0_magic", mag_df=mag_df
)

Rasterising claoaa_magic_meqpm2pyr...
Rasterising eno3_flux_meqpm2pyr...
Rasterising clminn_meqpm2pyr...
Rasterising clmaxnoaa_magic_meqpm2pyr...
Rasterising clmaxsoaa_magic_meqpm2pyr...
Rasterising anclimit_magic_ueqpl...
Rasterising anclimitoaa_magic_ueqpl...
Rasterising bc0_magic_ueqpl...
Rasterising clmins_meqpm2pyr...
Rasters saved to:
    /home/jovyan/shared/critical_loads/raster/magic_1860/water


## 3. Calculate exceedances

### 3.1. SSWC model

In [5]:
cl_fold = f"/home/jovyan/shared/critical_loads/raster/magic_{year}/water"
out_fold = f"/home/jovyan/shared/critical_loads/raster/magic_{year}/exceedance"
ex_df = cl.calculate_water_exceedance_sswc(
    ser_id, "1216", cl_fold, out_fold, cell_size=120, bc0="BC0_magic"
)
ex_df

Exceedance grid saved to:
    /home/jovyan/shared/critical_loads/raster/magic_1860/exceedance/sswc_ex_meqpm2pyr_1216_120m.tif


,series_id,medium,total_area_km2,exceeded_area_km2,exceeded_area_pct
0,28,water_sswc,322184,2101,1


### 3.2. FAB model

The code in this section calculates exceedances for water using the FAB model. This is based on my original code [here](https://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/critical_loads_workflow_new_grid.ipynb#2.3.3.-FAB-model).

In [6]:
# Constants to build paths
base_path = r"/home/jovyan/shared/critical_loads/raster"
short_name = "1216"

# Read CL arrays
array_dict = {}
for par in [
    "clminn_meqpm2pyr",
    "clmins_meqpm2pyr",
    "clmaxnoaa_magic_meqpm2pyr",
    "clmaxsoaa_magic_meqpm2pyr",
]:
    # Read tif
    tif_path = os.path.join(
        base_path, f"magic_{year}", "water", f"{par}_{cell_size}m.tif"
    )
    data, ndv, epsg, extent = nivapy.spatial.read_raster(tif_path)

    # Set NDV
    data[data == ndv] = np.nan

    # Add to dict
    array_dict[par] = data

# Read dep arrays
for par in ["ndep_mgpm2pyr", "sdep_mgpm2pyr"]:
    # Read tif
    tif_path = os.path.join(
        base_path, "deposition", f"{par}_{short_name}_{cell_size}m.tif"
    )
    data, ndv, epsg, extent = nivapy.spatial.read_raster(tif_path)
    data = data.astype(np.float32)

    # Set NDV
    data[data == ndv] = np.nan

    # Add to dict
    array_dict[par] = data

**The code below needs to be run on a "high memory" machine, even with 120 m resolution.**

In [7]:
%%time
# Extract arrays from dict
cln_min = array_dict["clminn_meqpm2pyr"]
cln_max = array_dict["clmaxnoaa_magic_meqpm2pyr"]
cls_min = array_dict["clmins_meqpm2pyr"]
cls_max = array_dict["clmaxsoaa_magic_meqpm2pyr"]
dep_n = array_dict["ndep_mgpm2pyr"] / 14  # Convert to meq
dep_s = array_dict["sdep_mgpm2pyr"] * 2 / 32.06  # Convert to meq

# Estimate exceedances
ex_n, ex_s, reg_id = cl.vectorised_exceed_ns_icpm(
    cln_min, cln_max, cls_min, cls_max, dep_n, dep_s
)

# Get exceeded area
ex = ex_n + ex_s
ex_area = np.count_nonzero(ex > 0) * cell_size * cell_size / 1.0e6
nor_area = np.count_nonzero(~np.isnan(dep_s)) * cell_size * cell_size / 1.0e6
ex_pct = 100 * ex_area / nor_area

# Build df and tidy
ex_df = pd.DataFrame(
    {
        "exceeded_area_km2": ex_area,
        "total_area_km2": nor_area,
        "exceeded_area_pct": ex_pct,
    },
    index=[0],
)

ex_df = ex_df.round(0).astype(int)
ex_df["series_id"] = ser_id
ex_df["medium"] = "water_fab"

ex_df = ex_df[
    ["series_id", "medium", "total_area_km2", "exceeded_area_km2", "exceeded_area_pct"]
]

ex_df

CPU times: user 12.2 s, sys: 4.67 s, total: 16.8 s
Wall time: 16.8 s


,series_id,medium,total_area_km2,exceeded_area_km2,exceeded_area_pct
0,28,water_fab,322184,14306,4


The code below writes the FAB results to GeoTiff format.

In [8]:
# Snap tiff
snap_tif = f"/home/jovyan/shared/critical_loads/raster/blr_land_mask_{cell_size}m.tif"

# N
n_tif = f"/home/jovyan/shared/critical_loads/raster/magic_{year}/exceedance/fab_exn_meqpm2pyr_{short_name}_{cell_size}m.tif"
cl.write_geotiff(ex_n, n_tif, snap_tif, -1, gdal.GDT_Float32)

# S
s_tif = f"/home/jovyan/shared/critical_loads/raster/magic_{year}/exceedance/fab_exs_meqpm2pyr_{short_name}_{cell_size}m.tif"
cl.write_geotiff(ex_s, s_tif, snap_tif, -1, gdal.GDT_Float32)

# N+S
ns_tif = f"/home/jovyan/shared/critical_loads/raster/magic_{year}/exceedance/fab_exns_meqpm2pyr_{short_name}_{cell_size}m.tif"
cl.write_geotiff(ex_n + ex_s, ns_tif, snap_tif, -1, gdal.GDT_Float32)

# Exceedance 'region'
reg_tif = f"/home/jovyan/shared/critical_loads/raster/magic_{year}/exceedance/fab_ex_reg_id_{short_name}_{cell_size}m.tif"
cl.write_geotiff(reg_id, reg_tif, snap_tif, -1, gdal.GDT_Float32)